# LangChain Function Call Tagging

From: https://learn.deeplearning.ai/functions-tools-agents-langchain/lesson/5/tagging-and-extraction

In [1]:
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
import json

## Use Pydantic (a validation library) to Define Functions

Pydantic is just a validation library. But LangChain uses it (along with its `convert_pydantic_to_openai_function` function) to make it easier to define function definitions that adheres to OpenAI function calling format.

In [2]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""
    sentiment: str = Field(description="sentiment of text, should be `pos`, `neg`, or `neutral`")
    language: str = Field(description="language of text (should be ISO 639-1 code)")

tagging_function = convert_pydantic_to_openai_function(Tagging)

print(json.dumps(tagging_function, indent=4))

{
    "name": "Tagging",
    "description": "Tag the piece of text with particular info.",
    "parameters": {
        "title": "Tagging",
        "description": "Tag the piece of text with particular info.",
        "type": "object",
        "properties": {
            "sentiment": {
                "title": "Sentiment",
                "description": "sentiment of text, should be `pos`, `neg`, or `neutral`",
                "type": "string"
            },
            "language": {
                "title": "Language",
                "description": "language of text (should be ISO 639-1 code)",
                "type": "string"
            }
        },
        "required": [
            "sentiment",
            "language"
        ]
    }
}


In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(temperature=0)
tagging_functions = [tagging_function]

prompt = ChatPromptTemplate.from_messages([
    ("system", "Think carefully, and then tag the text as instructed"),
    ("user", "{input}")
])

model_with_functions = model.bind(
    functions=tagging_functions,
    function_call={"name": "Tagging"} # Instruct OpenAI to always respond with function call using Tagging
)

## Create a Chain

In [4]:
tagging_chain = prompt | model_with_functions

In [5]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "pos",\n  "language": "en"\n}', 'name': 'Tagging'}})

In [6]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "neg",\n  "language": "it"\n}', 'name': 'Tagging'}})

## Parse Out Relevant Information

That `additional_kwargs` in the above output is ugly. We know the response will always be function_call b/c we forced it with `function_call={"name": "Tagging"}` and we know the arguments will always be JSON, so let's use `JsonOutputFunctionsParser` to just give us the final output in dict format.

In [7]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}